In [1]:
cd cmput_651_final/

In [2]:
%run init.ipynb

In [3]:
selected_worlds = get_selected()
world_name = 'den020d.map'
world = selected_worlds[world_name]

maps/selected
maps/selected/selected-map/lak110d.map
maps/selected/selected-map/den020d.map
maps/selected/selected-map/den005d.map
maps/selected/selected-map/ost102d.map
maps/selected/selected-map/den001d.map



In [4]:
roads = list(filter(lambda t: world.get(t) == World.ROAD, world))
idx_to_pos = dict(enumerate(roads))
vocab_size = len(idx_to_pos)
pos_to_idx = {v: k for k, v in idx_to_pos.items()}
print('vocab_size:', vocab_size)

vocab_size: 3102


In [5]:
def encode(pos):
    vec = np.zeros(vocab_size, dtype=np.int64)
    vec[pos_to_idx[pos]] = 1
    return vec

def decode(vec):
    return vec.argmax()

In [6]:
stream_dataset = StreamingPointsDataset(size=256, sample_rate=1)
batch_size = 1024 * 36

point_data_loader = torch.utils.data.DataLoader(
    stream_dataset,
    batch_size=batch_size,
    num_workers=8,
)

In [16]:
embed_dim = 512
hidden_dim = 512
model = LargeEmbedModel(vocab_size=vocab_size, embed_dim=embed_dim, hidden_dim=hidden_dim)
model.to('cpu')
summary(model, input_size=(2,), dtypes=[torch.int64], device='cpu')
model_name = f"{world_name}_{model.__class__.__name__}"

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Embedding-1               [-1, 2, 512]       1,588,224
            Linear-2                  [-1, 512]         524,800
           Dropout-3                  [-1, 512]               0
            Linear-4                 [-1, 3102]       1,591,326
   LargeEmbedModel-5                 [-1, 3102]               0
Total params: 3,704,350
Trainable params: 3,704,350
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 14.13
Estimated Total Size (MB): 14.19
----------------------------------------------------------------



In [18]:
optimizer = optim.Adam(model.parameters(), weight_decay=1e-8, lr=2e-3)
# criterion = F.mse_loss
criterion = F.nll_loss

In [19]:
model.train()
model.to(device)
sample_count = 0
for epoch in tqdm_nb(range(100)):
    losses = []
    for batch_idx, (x, y) in enumerate(point_data_loader):
        sample_count += x.shape[0]
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        output = model(x)
#         print(output.shape)
#         print(y.shape)
        loss = criterion(output, y.reshape(-1))
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    torch.save(model, f'{model_name}.pt')
    print('epoch: ', epoch,
          '\tloss: ',
          '\t', np.sum(np.array(losses)),
          '\t', np.mean(np.array(losses)),
#           '\t', np.mean(np.array(losses)) / batch_size,
#           '\t', np.mean(np.array(losses)) / len(point_data_loader),
         )
print('sample_count:', sample_count)
model.to('cpu')

epoch:  0 	loss:  	 57.45550727844238 	 7.181938409805298
epoch:  1 	loss:  	 48.948280811309814 	 6.118535101413727
epoch:  2 	loss:  	 45.19174242019653 	 5.648967802524567
epoch:  3 	loss:  	 43.022512912750244 	 5.3778141140937805
epoch:  4 	loss:  	 40.73072290420532 	 5.091340363025665
epoch:  5 	loss:  	 38.145461559295654 	 4.768182694911957
epoch:  6 	loss:  	 34.76305294036865 	 4.3453816175460815
epoch:  7 	loss:  	 31.786727905273438 	 3.9733409881591797
epoch:  8 	loss:  	 29.49608540534973 	 3.6870106756687164
epoch:  9 	loss:  	 26.90475082397461 	 3.363093852996826
epoch:  10 	loss:  	 24.91970133781433 	 3.1149626672267914
epoch:  11 	loss:  	 22.442652702331543 	 2.805331587791443
epoch:  12 	loss:  	 20.424217462539673 	 2.553027182817459
epoch:  13 	loss:  	 19.082897424697876 	 2.3853621780872345
epoch:  14 	loss:  	 18.004266262054443 	 2.2505332827568054
epoch:  15 	loss:  	 17.215227365493774 	 2.151903420686722
epoch:  16 	loss:  	 16.47668218612671 	 2.0595852

LargeEmbedModel(
  (embed): Embedding(3102, 512)
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (do1): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=512, out_features=3102, bias=True)
)

In [10]:
print(model_name)

den020d.map_LargeEmbedModel


In [11]:
torch.save(model, f'{world_name}_{str(model.__class__.__name__)}.pt')

In [12]:
world